In [ ]:
import torch
from datasets import load_dataset

DATASET_NAME = "DiscoResearch/germanrag"

dataset = load_dataset(DATASET_NAME, split="train")

In [145]:
print(dataset[0].keys())

dict_keys(['contexts', 'question', 'answer', 'positive_ctx_idx'])


In [ ]:
from tqdm.autonotebook import tqdm
# find unique contexts
unique_contexts = set()
for _, d in tqdm(enumerate(dataset)):
    for ctx in d["contexts"]:
        unique_contexts.add(ctx)

In [124]:
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import chroma
# need to load a german model actually.
from langchain_community.embeddings import HuggingFaceEmbeddings

text_splitter = CharacterTextSplitter()
chunks = [text_splitter.split_text(uc) for uc in unique_contexts]

In [78]:
hf_embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/distiluse-base-multilingual-cased-v2",
    model_kwargs = {'device': 'cuda'}
)

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.69k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/610 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [79]:
db = Chroma.from_texts(list(unique_contexts), hf_embedding, persist_directory=".")

In [80]:
answer = dataset[0]["answer"]
answer

'In Deutschland glauben 75 % der befragten Katholiken und 67 % der Protestanten an einen Gott.'

In [127]:
relevant_docs = db.similarity_search_with_relevance_scores(answer, k=25)

In [128]:
relevant_docs

[(Document(page_content='Straubing\n\n=== Konfessionsstatistik ===\nGemäß der Volkszählung 2011 waren 13,7 % der Einwohner evangelisch, 63,9 % römisch-katholisch und 22,4 % waren konfessionslos, gehörten einer anderen Religionsgemeinschaft an oder machten keine Angabe. Die Zahl der Protestanten und vor allem die der Katholiken ist seitdem gesunken. Laut Statistischem Bericht 2019 der Stadt Straubing waren Ende 2018 56,4 % der Bevölkerung römisch-katholisch, 10,7 % evangelisch und 32,9 % gehörten einer anderen oder keiner Glaubensgemeinschaft an. Die Mitgliederzahlen der Konfessionen erreichten 2019 einen neuen Tiefpunkt. Die Zahl der Katholiken war erstmals unter 55 % gesunken und 10,4 % der Straubinger waren evangelisch.'),
  0.46798024695328766),
 (Document(page_content='Paris\n\n=== Religionen ===\nEtwa 65 % der Einwohner sind getauft, rund 60 % bekennen sich zum römisch-katholischen Glauben, die meisten praktizieren den lateinischen Ritus, einige wenige auch den armenischen und ukr

In [141]:
threshed = db.similarity_search_with_relevance_scores(answer, k=25, score_threshold=0.35)
threshed

[(Document(page_content='Straubing\n\n=== Konfessionsstatistik ===\nGemäß der Volkszählung 2011 waren 13,7 % der Einwohner evangelisch, 63,9 % römisch-katholisch und 22,4 % waren konfessionslos, gehörten einer anderen Religionsgemeinschaft an oder machten keine Angabe. Die Zahl der Protestanten und vor allem die der Katholiken ist seitdem gesunken. Laut Statistischem Bericht 2019 der Stadt Straubing waren Ende 2018 56,4 % der Bevölkerung römisch-katholisch, 10,7 % evangelisch und 32,9 % gehörten einer anderen oder keiner Glaubensgemeinschaft an. Die Mitgliederzahlen der Konfessionen erreichten 2019 einen neuen Tiefpunkt. Die Zahl der Katholiken war erstmals unter 55 % gesunken und 10,4 % der Straubinger waren evangelisch.'),
  0.46798024695328766),
 (Document(page_content='Paris\n\n=== Religionen ===\nEtwa 65 % der Einwohner sind getauft, rund 60 % bekennen sich zum römisch-katholischen Glauben, die meisten praktizieren den lateinischen Ritus, einige wenige auch den armenischen und ukr

In [167]:
def find_easy_negs(data_instance, vector_store, n=1):
    reference = data_instance["answer"]
    contexts = set(data_instance["contexts"]) # faster in lookup
    
    relevant_docs = vector_store.similarity_search_with_relevance_scores(reference, k=20)
    
    # filter out docs which are in contexts
    relevant_docs = [doc for doc in relevant_docs if doc[0].page_content not in contexts]
    
    
    scores = torch.tensor([d[1] for d in relevant_docs])
    probas = scores.softmax(dim=-1)
    sampled = torch.multinomial(probas, num_samples=n)
    
    idx = sampled.item()
    
    return relevant_docs[idx][0].page_content

find_easy_negs(dataset[0], db)


"Strict_Baptists\nDie Strict Baptists (deutsch etwa: strikte, strenge Baptisten) sind eine lose Vereinigung von Baptisten mit etwa 6000 Mitgliedern in Großbritannien. Manchmal wird die Bezeichnung ''Gospel Standard Baptists'' verwendet. Die einzelnen Gemeinden sind relativ selbständig, so dass es auch gewisse Unterschiede in Lehre und Praxis geben kann. Sie gehören zur calvinistischen Richtung der ''Particular Baptists'', die im Unterschied zu den ''General Baptists'' nicht von einem universalen Heilswillen Gottes ausgehen. Die gemeinsame Feier des Herrenmahls findet in geschlossener Form statt (''strict'' oder ''closed communion''), d.\xa0h. nur die eigenen, getauften Mitglieder sind zugelassen. Diese ursprünglich sehr verbreitete enge Praxis wurde auch noch von Johann Gerhard Oncken (in Deutschland) aufrechterhalten, während Charles Haddon Spurgeon eine calvinistische Heilslehre mit einer offenen Herrenmahlpraxis verband. Ähnlich öffneten sich im 19. Jahrhundert viele Baptistengemein